#### Imports

In [1]:
# Telegram Bot
import telebot
import difflib
from telebot.types import InlineKeyboardMarkup, InlineKeyboardButton
from datetime import datetime, timedelta

import re
import spacy

from ingressos.ibge import get_estados_cidades
import time
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC

# Config Driver
from config.config_driver import ConfigDriver


#### Variaveis

In [2]:
API_TOKEN = '7135171388:AAExkHE82hR68W21q-KJCsxpJmVU9gtLNPA'
bot = telebot.TeleBot(API_TOKEN)

config_driver = ConfigDriver()
driver = config_driver.get_driver()

saudacoes = ['oi', 'olá', 'ola', 'eae', 'e aí', 'oiie', 'oie, tudo bem?', 'oi, tudo bem?', 'oi, como vai?', 'oi, como você está?']


#### Main

In [3]:
cinemas = None
user_text = None	

nlp = spacy.load("pt_core_news_sm")

def process_natural_language(message):
    global user_text
    user_text = message.text.lower().strip()
    doc = nlp(user_text)
    
    intent = None
    cidade = None
    date = None

    if any(token.lemma_ in ["cinema", "procurar", "buscar", "filme", "assistir"] for token in doc):
        intent = "cinema"
    
    city_match = re.search(r"em (.+)", user_text)
    if city_match:
        cidade = city_match.group(1).strip()
    
    if any(token.lemma_ in ["hoje"] for token in doc):
        date = datetime.today().strftime("%d/%m")
        
    if any(token.lemma_ in ["amanhã"] for token in doc):
        date = (datetime.today() + timedelta(days=1)).strftime("%d/%m")
    
    # Executando a ação com base na intenção e na cidade
    if intent == "cinema" and cidade:
        process_city_name(message, filmes=False, city=cidade, date=date)
    elif intent == "cinema":
        handle_cinema_markup(message)
    elif any(token.lemma_ in ["olá", "oi", "ajuda", "bom"] for token in doc):
        handle_first_contact(message)
    else:
        bot.send_message(message.chat.id, "Desculpe, não entendi. Tente frases como 'quero ver cinemas em [cidade]' ou 'buscar filmes em [cidade]'.")

@bot.message_handler(func=lambda message: True)
def reply_message(message):
    process_natural_language(message)

def handle_first_contact(message):
    bot.send_message(message.chat.id, "Olá, eu sou o bot do Ingresso.com, como posso te ajudar")

def handle_cinema_markup(message):
    state_msg = bot.send_message(message.chat.id, "Digite a cidade do cinema que deseja buscar")
    bot.register_next_step_handler(state_msg, process_city_name)

def process_city_name(message, filmes=False, city=None, date=None):
    if city is None:
        city = message.text.strip()

    # estados_cidades = get_estados_cidades()
    # todas_cidades = [cidade for cidades in estados_cidades.values() for cidade in cidades]
    # closest_matches = difflib.get_close_matches(city, todas_cidades, n=1, cutoff=0.6)
    closest_matches = "sdasa"

    if closest_matches:
        # city = closest_matches[0]
        city = "São José dos Campos"
        bot.send_message(message.chat.id, f"Buscando em {city}...")

        formatted_city = city.replace(" ", "-").lower()

        linkWebDriver = f"https://www.ingresso.com/cinemas?city={formatted_city}"
        driver.get(linkWebDriver)

        just_film = check_if_film_and_cinema_is_in_text(driver)

        print("Just film é igual a: " + str(just_film))

        if just_film:
            time.sleep(3)
            driver.get(f"https://www.ingresso.com/filmes?city={formatted_city}")
            time.sleep(3)
            
            films_titles = driver.find_elements(By.TAG_NAME, 'h4')

            for title in films_titles:
                print(title.text)
                if find_partial_match(title.text, user_text, 0.5):   
                    print("Achamos o filme")
                    time.sleep(4)
                    driver.execute_script("arguments[0].scrollIntoView({block: 'center'});", title)
                    time.sleep(1)
                    title.click()
                    print("Filme encontrado")
                    process_film_page(driver, message)
                    return
                
        linkWebDriver = f"https://www.ingresso.com/cinemas?city={formatted_city}"
        driver.get(linkWebDriver)
        try:
            fecharPopup = WebDriverWait(driver, 3).until(
                EC.presence_of_element_located((By.CLASS_NAME, "accept-btn"))
            )
            if fecharPopup.is_displayed():
                fecharPopup.click()
        except Exception:
            print("Popup nao encontrado.")

        get_cinemas_from_city(driver, message, date)
        
    else:
        bot.send_message(message.chat.id, "Nenhuma cidade correspondente encontrada. Tente novamente.")
        handle_cinema_markup(message)
    
def check_if_film_and_cinema_is_in_text(driver):
    global cinemas 
    try:
        fecharPopup = WebDriverWait(driver, 10).until(
            EC.presence_of_element_located((By.CLASS_NAME, "accept-btn"))
        )
        if fecharPopup.is_displayed():
            fecharPopup.click()
    except Exception:
        print("Popup nao encontrado. Ao checar se o cinema foi enviado no texto")
    cinemas = driver.find_elements(By.CSS_SELECTOR, "h3.text-base.leading-4")  

    ## exclui oq tem depois do em no user text
    texto_sem_cidade = re.sub(r"em.*", "", user_text)

    for cinema in cinemas:
        if find_partial_match(cinema.text, texto_sem_cidade):
            print("Cinema encontrado no texto")	
            return False
    
    print("Cinema não encontrado no texto")
    return True
    
def process_film_page(driver, message):
    print("Processing the movie page...")

    try:
        time.sleep(3)
        
        cinemas_container = driver.find_element(By.CSS_SELECTOR, ".mx-3.my-5")
        
        cinemas = cinemas_container.find_elements(By.CSS_SELECTOR, ".bg-ing-neutral-600")
        print(f"Encontrados {len(cinemas)} cinemas")
        for cinema in cinemas:
            nome_cinema = cinema.find_element(By.CSS_SELECTOR, ".text-ing-neutral").text
            print(nome_cinema)

            # Capturar o endereço no <span class="text-sm">
            endereco_element = cinema.find_elements(By.CSS_SELECTOR, "span.text-sm")
            endereco = endereco_element[0].text if endereco_element else "Endereço não encontrado"

            horarios_filmes = f"*Cinema:* {nome_cinema} \n "

            # Encontra todos os tipos de sessão
            tipo_sessao = cinema.find_elements(By.CSS_SELECTOR, ".mb-6")
            
            for tipo in tipo_sessao:
                # Encontra todas as informações dentro do tipo de sessão
                infos = tipo.find_elements(By.CSS_SELECTOR, ".uppercase.py-1")
                for info in infos:
                    horarios_filmes += f"*{info.text}* "
                
                # Encontra todos os elementos de horário dentro do tipo de sessão
                horarios_elements = tipo.find_elements(By.CSS_SELECTOR, "a[href*='sessionId']")
                horarios = []
                for horario_element in horarios_elements:
                    horario = horario_element.find_element(By.CSS_SELECTOR, "span").text  # Extrai o texto do horário
                    link = horario_element.get_attribute("href")  # Extrai o link
                    horarios.append((horario, link))  # Adiciona à lista de horários
                
                if len(horarios) == 0:
                    horarios_filmes += "\n Todas sessões esgotadas\n"
                else:
                    horarios_filmes += "\n Horários: \n"	
                
                for horario, link in horarios:
                    horarios_filmes += f"[{horario}]({link})\n"
              
                    
            bot.send_message(message.chat.id, horarios_filmes, parse_mode='MarkdownV2')
            
    except Exception as e:
        print(f"Erro ao processar um cinema: {e}")
            
def find_partial_match(keyword, user_text, cutoff=0.6):
    # Dividir o user_text em palavras
    words = user_text.split()
    
    # Percorrer todas as subseções do user_text que têm o mesmo comprimento que keyword
    for i in range(len(words) - len(keyword.split()) + 1):
        # Criar uma subseção do user_text com o mesmo número de palavras que keyword
        segment = ' '.join(words[i:i + len(keyword.split())])
        
        # Verificar a similaridade entre o keyword e a subseção
        similarity = difflib.SequenceMatcher(None, keyword.lower(), segment.lower()).ratio()
        
        # Retornar verdadeiro se a similaridade atingir o cutoff
        if similarity >= cutoff:
            return True
    
    return False

def find_partial_match_with_similarity(keyword, user_text, cutoff=0.6):
    words = user_text.split()
    best_match = None
    highest_similarity = 0
    
    for i in range(len(words) - len(keyword.split()) + 1):
        segment = ' '.join(words[i:i + len(keyword.split())])
        similarity = difflib.SequenceMatcher(None, keyword.lower(), segment.lower()).ratio()
        
        if similarity >= cutoff and similarity > highest_similarity:
            highest_similarity = similarity
            best_match = segment
    
    return highest_similarity, best_match

def get_cinemas_from_city(driver , message, date):
    print("Buscando cinemas da cidade")
    
    try:
        fecharPopup = WebDriverWait(driver, 3).until(
            EC.presence_of_element_located((By.CLASS_NAME, "accept-btn"))
        )
        if(fecharPopup.is_displayed()):
            fecharPopup.click()
    except Exception:
        print("Popup nao encontrado ao buscar os cinemas.")
    
    global cinemas 
    cinemas = driver.find_elements(By.CSS_SELECTOR, "h3.text-base.leading-4") 
    global user_text
    
    best_match_index = -1
    highest_similarity = 0
    
    for index, cinema in enumerate(cinemas):
        print("Buscando cinemas parecidos" + cinema.text)
        similarity, _ = find_partial_match_with_similarity(cinema.text, user_text)
        
        if similarity > highest_similarity:
            highest_similarity = similarity
            best_match_index = index

    if best_match_index != -1:
        print("Cinema encontrado")
        get_filmes_by_cinema(driver, best_match_index, message, date)
    else:
        print("Nenhum cinema encontrado")
    
    
    # Adicionar botões ao teclado com o texto do cinema e dados de callback
    markup = InlineKeyboardMarkup()
    for index, cinema in enumerate(cinemas):
        try:
            fecharPopup = WebDriverWait(driver, 3).until(
                EC.presence_of_element_located((By.CLASS_NAME, "accept-btn"))
            )
            if(fecharPopup.is_displayed()):
                fecharPopup.click()
        except Exception:
            print("Popup nao encontrado.")
    
        if(date == None):
            markup.add(InlineKeyboardButton(cinema.text, callback_data="ERIKAOXD_" + str(index)))
        else:
            markup.add(InlineKeyboardButton(cinema.text, callback_data="ERIKAOXD_" + str(index) + "_" + date))
        
    # Enviar o teclado inline para o usuário
    bot.send_message(message.chat.id, "Cinemas:", reply_markup=markup)

def get_filmes_by_cinema(driver, cinemaSelecionado, message, date=None):    
    global cinemas
    global user_text
    
    print(date)
    
    try:
        fecharPopup = WebDriverWait(driver, 10).until(
            EC.presence_of_element_located((By.CLASS_NAME, "accept-btn"))
        )
        if(fecharPopup.is_displayed()):
            fecharPopup.click()
    except Exception:
        print("Popup nao encontrado.")
    
    cinema_element = cinemas[cinemaSelecionado].find_element(By.XPATH, '..')

    # Rolagem para o cinema selecionado
    driver.execute_script("arguments[0].scrollIntoView({block: 'center'});", cinema_element)
    time.sleep(2)
    driver.execute_script("arguments[0].scrollIntoView({block: 'center'});", cinema_element)
    
    time.sleep(2)
    # Espera até que o elemento esteja visível e clicável
    WebDriverWait(driver, 10).until(EC.visibility_of(cinema_element))
    WebDriverWait(driver, 10).until(EC.element_to_be_clickable(cinema_element))

    cinema_element.click()

    if date == None:
        date = datetime.today().strftime("%d/%m")
    
    try:
        diaEscolhidoElemento = driver.find_element(By.XPATH, f"//span[contains(text(), '{date}')]")
    except Exception:
        time.sleep(3)
        try:
            diaEscolhidoElemento = driver.find_element(By.XPATH, f"//span[contains(text(), '{date}')]")
            driver.execute_script("arguments[0].scrollIntoView({block: 'center'});", diaEscolhidoElemento)
        except Exception:
            print(f"Element still not found for date: {date} after retrying")
    
    
    try:
        cookie_consent = driver.find_element(By.CLASS_NAME, "CookieConsent")
        if cookie_consent.is_displayed():
            close_button = cookie_consent.find_element(By.TAG_NAME, "button")
            close_button.click()
    except Exception:
        print("Cookie consent not found, continuing without clicking.")

    # Now attempt to click the desired element
    try:
        diaEscolhidoElemento = driver.find_element(By.XPATH, f"//span[contains(text(), '{date}')]")
        driver.execute_script("arguments[0].scrollIntoView({block: 'center'});", diaEscolhidoElemento)
        diaEscolhidoElemento.click()
    except Exception:
        print("Element click intercepted, please check the page layout.")


    # Captura os blocos de filmes
    blocos_filmes = driver.find_elements(By.CSS_SELECTOR, '.relative.my-5')
    
    
    for bloco in blocos_filmes:
        if(find_partial_match(bloco.find_element(By.CSS_SELECTOR, "h3 a").text, user_text)):
            process_and_send_movie_info(bloco, message)
            return
    
    # Itera sobre cada bloco para extrair o nome do filme, horários e links
    bot.send_message(message.chat.id, "Filmes em cartaz:")
    for bloco in blocos_filmes:
        process_and_send_movie_info(bloco, message)

def process_and_send_movie_info(bloco, message):
    # Extrai o nome do filme
    nome_filme = bloco.find_element(By.CSS_SELECTOR, "h3 a").text
    
    horarios_filmes = f"*Filme:* {nome_filme}\n\n"
    
    # Encontra todos os tipos de sessão
    tipo_sessao = bloco.find_elements(By.CSS_SELECTOR, ".mx-0.mt-0")
    
    for tipo in tipo_sessao:
        # Encontra todas as informações dentro do tipo de sessão
        infos = tipo.find_elements(By.CSS_SELECTOR, ".inline-block.py-1")
        for info in infos:
            horarios_filmes += f"*{info.text}* "
        
        # Encontra todos os elementos de horário dentro do tipo de sessão
        horarios_elements = tipo.find_elements(By.CSS_SELECTOR, "a[href*='sessionId']")
        horarios = []
        for horario_element in horarios_elements:
            horario = horario_element.find_element(By.CSS_SELECTOR, "span").text  # Extrai o texto do horário
            link = horario_element.get_attribute("href")  # Extrai o link
            horarios.append((horario, link))  # Adiciona à lista de horários
        
        if len(horarios) == 0:
            horarios_filmes += "\n Todas sessões esgotadas\n"
        else:
            horarios_filmes += "\n Horários: \n"
        
        for horario, link in horarios:
            horarios_filmes += f"[{horario}]({link})\n"

    imgFilme = bloco.find_element(By.CSS_SELECTOR, "img").get_attribute("src")
    horarios_filmes.replace("-", " ")

    # Enviar a foto e a mensagem para o usuário
    bot.send_photo(message.chat.id, imgFilme, parse_mode='MarkdownV2')
    bot.send_message(message.chat.id, horarios_filmes, parse_mode='MarkdownV2')


        
@bot.callback_query_handler(func=lambda call: True)
def callback_query(call):
    if call.data.startswith("ERIKAOXD"):
        index = int(call.data.split("_")[1])
        date = call.data.split("_")[2] if len(call.data.split("_")) > 2 else None
        get_filmes_by_cinema(driver, index, message=call.message, date=date)
    else:
        # Outros callbacks para "cinema" ou "filme"
        if call.data == "cinema":
            handle_cinema_markup(call.message)


#### Run Bot

In [4]:
bot.polling()

Cinema encontrado no texto
Just film é igual a: False
Popup nao encontrado.
Buscando cinemas da cidade
Popup nao encontrado ao buscar os cinemas.
Buscando cinemas parecidosCinemark Center Vale
Buscando cinemas parecidosCinemark Colinas
Buscando cinemas parecidosCinépolis Jardim Oriente
Buscando cinemas parecidosKinoplex Diamante
Buscando cinemas parecidosKinoplex Vale Sul
Cinema encontrado
None
Popup nao encontrado.
Element click intercepted, please check the page layout.
Popup nao encontrado.


StaleElementReferenceException: Message: stale element reference: stale element not found in the current frame
  (Session info: MicrosoftEdge=131.0.2903.51); For documentation on this error, please visit: https://www.selenium.dev/documentation/webdriver/troubleshooting/errors#stale-element-reference-exception
Stacktrace:
	GetHandleVerifier [0x00007FF65644DC75+12853]
	Microsoft::Applications::Events::EventProperty::empty [0x00007FF6566F83C4+2250276]
	Microsoft::Applications::Events::EventProperty::empty [0x00007FF6566376A6+1460486]
	(No symbol) [0x00007FF65620B058]
	(No symbol) [0x00007FF65620CC57]
	(No symbol) [0x00007FF65620CD10]
	(No symbol) [0x00007FF65624558B]
	(No symbol) [0x00007FF6562684FA]
	(No symbol) [0x00007FF656240617]
	(No symbol) [0x00007FF6562404DD]
	(No symbol) [0x00007FF6562687E0]
	(No symbol) [0x00007FF656240617]
	(No symbol) [0x00007FF656281431]
	(No symbol) [0x00007FF656268163]
	(No symbol) [0x00007FF65623FB54]
	(No symbol) [0x00007FF65623F000]
	(No symbol) [0x00007